In [1]:
# importando bibliotecas
import gurobipy as gp
from gurobipy import GRB

# Capacitated facility location problem

Uma companhia entrega diariamente seus produtos que são produzidos em 5 fábricas para 4 armazens. 

Nesse processo de entrega existem dois tipos de custos, o custo associado ao transporte e o custo fixo de uso da fábrica. 

Cada fábrica possui um limite quanto a quantidade de produtos a serem entregues. A companhia estudar fechar algumas fábricas para reduzir custos. 

Quais fábricas a companhia deve fechar, com a finalidade de reduzir os custos?

In [2]:
# demanda dos armazéns
demand = [15, 18, 14, 20]

# capacidade de produção da cada fábrica
capacity = [20, 22, 17, 19, 18]

# custo fixo de cada fábrica
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# custo de transporte
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

# quantidade de fábricas
plants = range(len(capacity))

# quantidade de armazéns
warehouses = range(len(demand))

In [7]:
# criando modelo
model = gp.Model("capfacility")

# y[p]: y[p] == 1 se a fábrica p é aberta, 0 caso contrário.
y = model.addVars(plants,vtype=GRB.BINARY,obj=fixedCosts,name="y")

# x[w,p]: quantidade transportada da fábrica p para o armazém i
x = model.addVars(warehouses, plants, obj=transCosts, name="x")

# sentido da função objetivo
model.modelSense = GRB.MINIMIZE

# restrição de produção
model.addConstrs((x.sum('*', p) <= capacity[p] * y[p] for p in plants), "cap")

# restrição de demanda
model.addConstrs((x.sum(w,'*') == demand[w] for w in warehouses), "dem")

# salva o modelo
model.write('capfacility.lp')

# usa o método barrier para resolver o relaxação linear na raiz
model.Params.method = 2

# resolve o problema
model.optimize()

Changed value of parameter method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 25 columns and 45 nonzeros
Model fingerprint: 0xb5a24dbe
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.94290841e+05 -2.24842916e+05  7.25e+00 3.75e+03  

In [8]:
# imprime a solução
print('\nCusto total: %g' % model.objVal)

print('\n Solução:')
for p in plants:
    if y[p].x > 0.99:
        print('\nFábrica %s é aberto' % p)
        for w in warehouses:
            if x[w, p].x > 0:
                print('Transporte de %g unidades para o armazém %s' % (x[w, p].x, w))
    else:
        print('\nFábrica %s é fechada!' % p)


Custo total: 210500

 Solução:

Fábrica 0 é aberto
Transporte de 14 unidades para o armazém 2
Transporte de 6 unidades para o armazém 3

Fábrica 1 é aberto
Transporte de 14 unidades para o armazém 0
Transporte de 8 unidades para o armazém 3

Fábrica 2 é fechada!

Fábrica 3 é aberto
Transporte de 1 unidades para o armazém 0
Transporte de 18 unidades para o armazém 1

Fábrica 4 é aberto
Transporte de 6 unidades para o armazém 3


In [9]:
# outro modelo

# quantidade de fábricas
plants = range(len(capacity))

# quantidade de armazéns
warehouses = range(len(demand))

# criando modelo
model1 = gp.Model("capfacility1")

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
#
# variável y: y[p] == 1 se a fábrica p é aberta, 0 caso contrário
y = []
for p in plants:
    y.append(model1.addVar(vtype=GRB.BINARY,obj=fixedCosts[p],name="y[%d]" % p))

# variável x: x[w,p] é quantidade transportada da fábrica p para o armazém w
x = []
for w in warehouses:
    x.append([])
    for p in plants:
        x[w].append(model1.addVar(obj=transCosts[w][p],name="x[%d,%d]" % (w, p)))

# problema de mininização
model1.modelSense = GRB.MINIMIZE

# restrição de produção
for p in plants:
    model1.addConstr(sum(x[w][p] for w in warehouses) <= 
                 capacity[p] * y[p], "cap[%d]" % p)

# restrição de demanda
for w in warehouses:
    model1.addConstr(sum(x[w][p] for p in plants) == demand[w], "dem[%d]" % w)

# escrevendo o modelo
model1.write('capfacility1.lp')

# usando o método barreira para resolver a relaxação na raiz
model1.Params.method = 2

# resolve o problema
model1.optimize()

# imprime a solução
print('\nCusto total: %g' % model1.objVal)

print('\n Solução:')
for p in plants:
    if y[p].x > 0.99:
        print('\nFábrica %s é aberto' % p)
        for w in warehouses:
            if x[w][p].x > 0:
                print('Transporte de %g unidades para o armazém %s' % (x[w][p].x, w))
    else:
        print('\nFábrica %s é fechada!' % p)

Changed value of parameter method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 25 columns and 45 nonzeros
Model fingerprint: 0xb5a24dbe
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.94290841e+05 -2.24842916e+05  7.25e+00 3.75e+03  